In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199775 sha256=aeef8c577825d9d3629c39cfba717a476269bab4c9df1eb2ffd4367376b5021e
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [ ]:
import pandas as pd
df=pd.read_csv('/content/drive/MyDrive/hate rate/task 4/dataset/train_AH_preprocessed.csv')

In [ ]:
import pandas as pd

# Assuming 'df' is your DataFrame
df = df.sample(frac=1).reset_index(drop=True)


In [ ]:
#convert label to int
output=pd.Series()
for indx,row in df.iterrows():
  current=row['task_1']
  if(current=="NOT"):
    output.at[indx]=0
  else:
    output.at[indx]=1
df['output']=output


#finding longest sentence
maxx=0

for s in df['text']:
  s_len=len(s.split())
  if(s_len>maxx):
    sent=s
  maxx=max(s_len,maxx)


print(maxx)
# Pad sequences to a fixed length (adjust MAX_LEN if needed)
MAX_LEN = maxx

<ipython-input-5-5acdf9bf36d9>:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  output=pd.Series()


66


In [ ]:
import fasttext

# Replace 'path_to_model.bin' with the actual path to your .bin file
fasttext_model = fasttext.load_model('/content/drive/MyDrive/hate rate/task 4/fasttext/assamese/cc.as.300.bin')


i think i dont need word_vector

In [ ]:
def load_word_vectors(file_path):
    word_vectors = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.rstrip().split(' ')
            word = values[0]
            vector = [float(val) for val in values[1:]]
            word_vectors[word] = vector
    return word_vectors

# Replace 'path_to_vectors.vec' with the actual path to your .vec file
word_vectors = load_word_vectors('/content/drive/MyDrive/hate rate/task 4/fasttext/assamese/cc.as.300.vec')


In [ ]:
import fasttext
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load your FastText model
#TODO add filters=' ' to tokenizer
# Create a tokenizer based on FastText vocabulary
tokenizer = Tokenizer(oov_token='<oov>')
tokenizer.fit_on_texts(df['text'].tolist())
# Add special tokens to the tokenizer

# Convert sentences to sequences of word indices
sequences = tokenizer.texts_to_sequences(df['text'].tolist())

# Pad sequences to a uniform length
max_sequence_length = 180  # Define your desired sequence length
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length, padding='post', truncating='post')

# Create the embedding matrix
embedding_dim = 300  # Adjust based on your FastText embeddings
num_words = len(tokenizer.word_index) + 1  # Add 1 for the padding token
count_yes=0
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in fasttext_model:
        count_yes+=1
        embedding_matrix[i] = fasttext_model[word]
    else:
        # Initialize random embedding for OOV words
        embedding_matrix[i] = np.random.uniform(-1, 1, embedding_dim)
# Now you can use 'embedding_matrix' as the 'weights' parameter when creating the Embedding layer in your model
print('number of words in fasttext of our tokenizer are : ',count_yes)

number of words in fasttext of our tokenizer are :  6786


In [ ]:
len(tokenizer.word_index)

11173

In [ ]:
embedding_matrix.shape

(11174, 300)

In [ ]:
train_input=padded_sequences
train_output=np.array(df['output'])

In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split






# Split the dataset into training and validation sets
#train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

# Convert the DataFrame to a TensorFlow dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_input, train_output))
##val_dataset = tf.data.Dataset.from_tensor_slices((val_df['seq'], val_df['output']))

# Shuffle and batch the datasets
batch_size = 32
train_dataset = train_dataset.shuffle(len(df)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
#val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Add, Input,Concatenate,Bidirectional,Dropout

# Assuming you have loaded FastText embeddings into 'embedding_matrix'

# Input layer
input_layer = Input(shape=(max_sequence_length,))

# Embedding layer
embedding_layer = Embedding(input_dim=num_words,
                            output_dim=embedding_dim,
                            weights=[embedding_matrix],
                            input_length=max_sequence_length,
                            trainable=True)(input_layer)

# 1D CNN layer with MaxPooling
# Convolutional layers with different filter lengths
conv4 = Conv1D(filters=128, kernel_size=32, activation='relu')(embedding_layer)
maxConv4 = MaxPooling1D(pool_size=2)(conv4)

conv5 = Conv1D(filters=128, kernel_size=32, activation='relu')(embedding_layer)
maxConv5 = MaxPooling1D(pool_size=2)(conv5)

# Concatenate the outputs of the Convolutional layers
residual_connection = Concatenate(axis=-1)([maxConv4, maxConv5])

# LSTM layer
lstm_layer = Bidirectional(LSTM(64))(residual_connection)

# Dense layer
dense_layer = Dense(64, activation='relu')(lstm_layer)
dense_layer=Dropout(0.5)(dense_layer)
dense_layer = Dense(32, activation='relu')(dense_layer)
# Output layer
output_layer = Dense(1, activation='sigmoid')(dense_layer)

# Create the model
model = tf.keras.Model(inputs=input_layer, outputs=output_layer)



# Print the model summary
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 180)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 180, 300)     3352200     ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 149, 128)     1228928     ['embedding[0][0]']              
                                                                                                  
 conv1d_1 (Conv1D)              (None, 149, 128)     1228928     ['embedding[0][0]']              
                                                                                              

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(train_input, train_output, test_size=0.1, random_state=22)

In [ ]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
BATCH_SIZE=8
EPOCHS=1200


  # You can adjust this value
learning_rate = 0.000001
momentum = 0.9  # Optional: If you want to use momentum

# Create the SGD optimizer
sgd_optimizer = SGD(learning_rate=learning_rate, momentum=momentum)
ad= Adam(learning_rate=learning_rate)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=ad, metrics=['accuracy'])


model.fit(x_train,
          y_train,
          batch_size=BATCH_SIZE,
          validation_data=(x_val,y_val),
          epochs=EPOCHS)

Epoch 1/1200
454/454 [==============================] - 46s 55ms/step - loss: 0.6935 - accuracy: 0.5052 - val_loss: 0.6937 - val_accuracy: 0.5025
Epoch 2/1200
454/454 [==============================] - 9s 20ms/step - loss: 0.6935 - accuracy: 0.4937 - val_loss: 0.6932 - val_accuracy: 0.5173
Epoch 3/1200
454/454 [==============================] - 9s 20ms/step - loss: 0.6927 - accuracy: 0.5374 - val_loss: 0.6928 - val_accuracy: 0.5272
Epoch 4/1200
454/454 [==============================] - 8s 19ms/step - loss: 0.6921 - accuracy: 0.5476 - val_loss: 0.6924 - val_accuracy: 0.5495
Epoch 5/1200
454/454 [==============================] - 7s 16ms/step - loss: 0.6920 - accuracy: 0.5471 - val_loss: 0.6920 - val_accuracy: 0.5693
Epoch 6/1200
454/454 [==============================] - 9s 19ms/step - loss: 0.6912 - accuracy: 0.5639 - val_loss: 0.6916 - val_accuracy: 0.5842
Epoch 7/1200
454/454 [==============================] - 8s 17ms/step - loss: 0.6909 - accuracy: 0.5699 - val_loss: 0.6912 - val_a

after 60 epoch there was a chnage in val accuracy from 60.40 to 60.64% i am testing for 60 more epochs


In [ ]:
r_num=99
x_train1, x_val1, y_train1, y_val1 = train_test_split(train_input, train_output, test_size=0.3, random_state=r_num)

In [ ]:
model.evaluate(x_val,y_val)

38/38 [==============================] - 0s 10ms/step - loss: 0.7655 - accuracy: 0.6210


[0.7655049562454224, 0.6209744215011597]

In [ ]:
#without early stopping we are getting 62%

In [ ]:
binar_class=np.where(output>0.5,1,0)

63% with 30% data trained fresh


## adding early stopping


In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
BATCH_SIZE = 8
EPOCHS = 30

learning_rate = 0.00001
momentum = 0.9

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(train_input, train_output, test_size=0.3, random_state=42)

ad= Adam(learning_rate=learning_rate)
# Compile the model
model.compile(loss='binary_crossentropy', optimizer=ad, metrics=['accuracy'])

# Define the EarlyStopping callback
# Define the EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=6, restore_best_weights=True)


# Train the model with early stopping
history = model.fit(x_train,
                    y_train,
                    batch_size=BATCH_SIZE,
                    validation_data=(x_val, y_val),
                    epochs=EPOCHS,
)


Epoch 1/30
354/354 [==============================] - 26s 57ms/step - loss: 0.6917 - accuracy: 0.5221 - val_loss: 0.6881 - val_accuracy: 0.5739
Epoch 2/30
354/354 [==============================] - 8s 23ms/step - loss: 0.6837 - accuracy: 0.5851 - val_loss: 0.6829 - val_accuracy: 0.5747
Epoch 3/30
354/354 [==============================] - 8s 24ms/step - loss: 0.6752 - accuracy: 0.5865 - val_loss: 0.6798 - val_accuracy: 0.5747
Epoch 4/30
354/354 [==============================] - 8s 24ms/step - loss: 0.6657 - accuracy: 0.5880 - val_loss: 0.6784 - val_accuracy: 0.5747


now training the model on k fold

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import Adam

# Assuming you have 'train_input' and 'train_output' data loaded

BATCH_SIZE = 32
EPOCHS = 3
K_FOLDS = 5  # Number of folds for cross-validation

# Create the Adam optimizer
learning_rate=0.00001
optimizer = Adam(learning_rate=learning_rate)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Initialize k-fold cross-validation
kf = KFold(n_splits=K_FOLDS, shuffle=True, random_state=42)

fold = 1
for train_index, val_index in kf.split(train_input):
    print(f"Fold {fold}/{K_FOLDS}")

    # Create a new model for each fold
    print('**************************reintializing****************')
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    # Get the train and validation data for this fold
    x_train, x_val = train_input[train_index], train_input[val_index]
    y_train, y_val = train_output[train_index], train_output[val_index]
    before_score=model.evaluate(x_val, y_val, verbose=0)
    print('before training the current validation score is ',before_score)
    # Train the model for this fold
    model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=15, validation_data=(x_val, y_val))

    # Evaluate the model on the validation set
    scores = model.evaluate(x_val, y_val, verbose=0)
    print(f"Fold {fold} - Validation loss: {scores[0]}, Validation accuracy: {scores[1]}")

    fold += 1


Fold 1/5
**************************reintializing****************
before training the current validation score is  [0.6940086483955383, 0.4653465449810028]
Epoch 1/15
101/101 [==============================] - 20s 137ms/step - loss: 0.6924 - accuracy: 0.5372 - val_loss: 0.6902 - val_accuracy: 0.5631
Epoch 2/15
101/101 [==============================] - 6s 53ms/step - loss: 0.6884 - accuracy: 0.5864 - val_loss: 0.6881 - val_accuracy: 0.5681
Epoch 3/15
101/101 [==============================] - 5s 47ms/step - loss: 0.6852 - accuracy: 0.5858 - val_loss: 0.6861 - val_accuracy: 0.5681
Epoch 4/15
101/101 [==============================] - 6s 62ms/step - loss: 0.6821 - accuracy: 0.5861 - val_loss: 0.6845 - val_accuracy: 0.5681
Epoch 5/15
101/101 [==============================] - 4s 42ms/step - loss: 0.6784 - accuracy: 0.5843 - val_loss: 0.6831 - val_accuracy: 0.5681
Epoch 6/15
101/101 [==============================] - 4s 37ms/step - loss: 0.6754 - accuracy: 0.5852 - val_loss: 0.6824 - val_ac

KeyboardInterrupt: ignored

with adam optimizer with 0.001 learning rate we are geting 60%


it is using adam opttimizer with unknown learning rate

In [ ]:
# Initialize k-fold cross-validation
kf = KFold(n_splits=K_FOLDS, shuffle=True)
for train_index, val_index in kf.split(train_input):
  scores = model.evaluate(x_val, y_val, verbose=0)
  print(f"Fold {fold} - Validation loss: {scores[0]}, Validation accuracy: {scores[1]}")

Fold 5 - Validation loss: 0.07562540471553802, Validation accuracy: 0.9851301312446594


KeyboardInterrupt: ignored

In [ ]:
model.evaluate(train_input,train_output)

127/127 [==============================] - 68s 536ms/step - loss: 0.5987 - accuracy: 0.8855


[0.5987412929534912, 0.8855302333831787]

In [ ]:
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import Callback
BATCH_SIZE=8
EPOCHS=100


initial_learning_rate = 0.0001
momentum = 0.9  # Optional: If you want to use momentum


class LearningRateSchedulerWithPatience(Callback):
    def __init__(self, initial_lr, patience, factor):
        super().__init__()
        self.initial_lr = initial_lr
        self.patience = patience
        self.factor = factor
        self.best_val_acc = -np.inf
        self.wait = 0

    def on_epoch_end(self, epoch, logs=None):
        val_acc = logs.get('val_accuracy')

        if val_acc > self.best_val_acc:
            self.best_val_acc = val_acc
            self.wait = 0
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.model.optimizer.lr = self.model.optimizer.lr * self.factor
                self.wait = 0
                print("Learning rate reduced:", self.model.optimizer.lr.numpy())

# Create the custom LearningRateSchedulerWithPatience callback
lr_scheduler = LearningRateSchedulerWithPatience(initial_lr=initial_learning_rate, patience=6, factor=0.9)







  # You can adjust this value
learning_rate = 0.0001
momentum = 0.9  # Optional: If you want to use momentum

# Create the SGD optimizer
sgd_optimizer = SGD(learning_rate=learning_rate, momentum=momentum)
# Compile the model
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='binary_crossentropy', optimizer=sgd_optimizer, metrics=['accuracy'])

model.fit(train_input,
          train_output,
          batch_size=BATCH_SIZE,
          validation_split=0.3,
          epochs=EPOCHS,
          callbacks=[lr_scheduler])

Epoch 1/100
354/354 [==============================] - 145s 397ms/step - loss: 0.6912 - accuracy: 0.5508 - val_loss: 0.6876 - val_accuracy: 0.5888
Epoch 2/100
354/354 [==============================] - 607s 2s/step - loss: 0.6876 - accuracy: 0.5681 - val_loss: 0.6833 - val_accuracy: 0.5904
Epoch 3/100
354/354 [==============================] - 97s 272ms/step - loss: 0.6848 - accuracy: 0.5777 - val_loss: 0.6802 - val_accuracy: 0.5912
Epoch 4/100
354/354 [==============================] - 7s 21ms/step - loss: 0.6835 - accuracy: 0.5766 - val_loss: 0.6785 - val_accuracy: 0.5912
Epoch 5/100
354/354 [==============================] - 7s 18ms/step - loss: 0.6815 - accuracy: 0.5752 - val_loss: 0.6774 - val_accuracy: 0.5912
Epoch 6/100
354/354 [==============================] - 8s 22ms/step - loss: 0.6811 - accuracy: 0.5763 - val_loss: 0.6769 - val_accuracy: 0.5912
Epoch 7/100
354/354 [==============================] - 7s 20ms/step - loss: 0.6811 - accuracy: 0.5766 - val_loss: 0.6766 - val_accu

this is first code that is able to decrase below 6800

In [ ]:
from tensorflow.keras.optimizers import SGD
BATCH_SIZE=8
EPOCHS=100


  # You can adjust this value
learning_rate = 0.00001
momentum = 0.9  # Optional: If you want to use momentum

# Create the SGD optimizer
sgd_optimizer = SGD(learning_rate=learning_rate, momentum=momentum)
# Compile the model
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='binary_crossentropy', optimizer=sgd_optimizer, metrics=['accuracy'])


model.fit(train_input,
          train_output,
          batch_size=BATCH_SIZE,
          validation_split=0.05,
          epochs=EPOCHS)